# **Récolte des données** 

### Modification à faire :
- **Pouvoir demander une liste de points d'interêts au lieu d'un seul**


- Retravailler les cellules markdown et les commentaires Python
- Restructurer les paramètres

### Importation des librairies utiles

In [166]:
import requests
import json
import pandas as pd
import overpy
import geopy.distance as gd

### Création du dictionnaire des Localisation avec leur URL respectives

In [167]:
#Localisation des données
dict_association_localisation = {"AGGLOMÉRATION LE COTENTIN":'https://vigilo.velorution-cherbourg.fr/get_issues.php?',
                                 "AGGLOMÉRATION TROYENNE":'https://vigilo.troyesenselle.fr/get_issues.php?', 
                                 "AIX MARSEILLE PROVENCE METROPOLE":'https://vigilo.velosenville.org/get_issues.php?',
                                 "BASSIN D'ARCACHON":'https://vigilo.velobassin.org/get_issues.php?',
                                 "BEAUVAISIS":'https://vigilo.vellovaque.fr/get_issues.php?',
                                 "BORDEAUX MÉTROPOLE":'https://vigilo.cycles-manivelles.org/get_issues.php?',
                                 "BREST MÉTROPOLE":'https://vigilo.bapav.org/get_issues.php?',
                                 "EVREUX":'https://vigilo.cube27.org/get_issues.php?',
                                 "GRAND MONTPELLIER":'https://vigilo.jesuisundesdeux.org/get_issues.php?',
                                 "LA ROCHELLE":'https://vigilo.-lr.corolo.fr/get_issues.php?',
                                 "LORIENT AGGLO":'https://vigilo.claav.fr/get_issues.php?',
                                 "MONS":'https://vigilo.geckonline.be/get_issues.php?',
                                 "NANCY AGGLOMÉRATION":'https://vigilo.rigelk.eu/get_issues.php?',
                                 "NANTES MÉTROPOLE":'https://vigilo.placeauvelo-nantes.fr/get_issues.php?',
                                 "NORD-ISÈRE (APIE)":'https://vigilo.apie-asso.net/get_issues.php?',
                                 "PARIS SACLAY":'https://api.vigilo-parissaclay.mdb-idf.org/get_issues.php?',
                                 "PAYS DE CHÂTEAUGIRON":'https://pouliquen.net/vigilo/get_issues.php?',
                                 "PÉRIGUEUX AGGLO":'https://vigilo.velorution-perigourdine.fr/get_issues.php?',
                                 "SAINT MARCELLIN VERCORS ISÈRE COMMUNAUTÉ":'https://vigilo.roulavelo.org/get_issues.php?',
                                 "SAINT-BRIEUC ARMOR AGGLOMÉRATION":'https://api-vigilo.velo-utile.fr/get_issues.php?',
                                 "STRASBOURG":'https://www.strasbourgavelo.eu/get_issues.php?',
                                 "SUD OISE":'https://vigilo.au5v.fr//get_issues.php?',
                                 "THAU":'https://prod-sete.vigilo.velocite-montpellier.fr/get_issues.php?',
                                 "VANNES":'https://vigilo.kaz.bzh/get_issues.php?'
                                }

df_localisation = pd.DataFrame(dict_association_localisation.keys(), columns=["Localisation"])
df_localisation.set_index("Localisation", inplace=True)
df_localisation

""
Localisation
AGGLOMÉRATION LE COTENTIN
AGGLOMÉRATION TROYENNE
AIX MARSEILLE PROVENCE METROPOLE
BASSIN D'ARCACHON
BEAUVAISIS
BORDEAUX MÉTROPOLE
BREST MÉTROPOLE
EVREUX
GRAND MONTPELLIER


### Création du dictionnaire avec les différentes catégories de signalements et leur numéro

In [168]:
# URL de l'API pour récupérer la liste des catégories
url_categories = 'https://vigilo-bf7f2.firebaseio.com/categorieslist.json'
resp = requests.get(url_categories)
resp.raise_for_status()

# Convertir la liste de catégories en un dictionnaire avec le nom de la catégorie comme clé et l'identifiant de catégorie comme valeur
categories_dict = {str(cat["catid"]): cat["catname"] for cat in resp.json()}

# Créer un dataframe à partir du dictionnaire
df_categories = pd.DataFrame(categories_dict.items(), columns=["id", "Catégories"])

# Changer l'index et trier par id
df_categories = df_categories.set_index("id")

# Afficher le dataframe
df_categories

,Catégories
id,
8,Absence d'aménagement
3,Aménagement mal conçu
4,Défaut d'entretien
5,Absence d'arceaux de stationnement
2,Véhicule ou objet gênant
9,"Accident, chute, incident"
6,"Signalisation, marquage"
7,Incivilité récurrente sur la route
10,Vol ou dégradation de vélo


### Liste des POI

In [169]:
# Créer un dictionnaire des points d'intérêt et leur identifiant OSM (ex: pour les parkings à vélo : "amenity:bicycle_parking")

poi_dict = {
    "Écoles": "amenity=school",
    "Hôpitaux": "amenity=hospital",
    "Parkings à vélos": "amenity=bicycle_parking",
    "Stations de vélos en libre-service": "amenity=bicycle_rental",
    "Passages piétons": "highway=crossing",
    "Feux piétons": "highway=traffic_signals",
    "Places": "highway=pedestrian",
    "Rampes d'accès pour handicapés": "amenity=wheelchair_ramp",
    "Sièges pour handicapés": "amenity=wheelchair_seating",
    "Parkings pour handicapés": "amenity=parking_space",
    "Parcs": "leisure=park",
    "Jardins": "leisure=garden",
    "Terrains de jeux": "leisure=playground",
    "Toilettes publiques": "amenity=toilets",
    "Fontaines à eau": "amenity=drinking_water",
    "Marchés": "amenity=marketplace",
    "Supermarchés": "shop=supermarket",
    "Boulangeries": "shop=bakery",
    "Pharmacies": "amenity=pharmacy",
    "Bibliothèques": "amenity=library",
    "Bureaux de poste": "amenity=post_office",
    "Stations de recharge pour véhicules électriques": "amenity=charging_station",
    "Cliniques": "amenity=clinic",
    "Dentistes": "amenity=dentist",
    "Vétérinaires": "amenity=veterinary",
    "Points de vue": "tourism=viewpoint",
    "Hôtels": "tourism=hotel",
    "Aires de pique-nique": "leisure=picnic_table",
    "Écoles maternelles": "amenity=kindergarten",
    "Écoles primaires": "amenity=primary_school",
    "Écoles secondaires": "amenity=secondary_school"
}

# Créer un dataframe à partir du dictionnaire
df_poi = pd.DataFrame(poi_dict.items(), columns=["Points d'intérêt", "Identifiant OSM"])

# Changer l'index
df_poi = df_poi.set_index("Identifiant OSM")

# Afficher le dataframe
df_poi

,Points d'intérêt
Identifiant OSM,
amenity=school,Écoles
amenity=hospital,Hôpitaux
amenity=bicycle_parking,Parkings à vélos
amenity=bicycle_rental,Stations de vélos en libre-service
highway=crossing,Passages piétons
highway=traffic_signals,Feux piétons
highway=pedestrian,Places
amenity=wheelchair_ramp,Rampes d'accès pour handicapés
amenity=wheelchair_seating,Sièges pour handicapés


### Initialisation des paramètres

- Choississez une localisation dans la liste
- Choississez une catégorie et entrez son identifiant (id)
- Choississez un ou plusieurs points d'intérêts
- Définissez la boîte de recherche, pour cela, choississez sur 'site' un point au centre de l'endroit qui vous intéresse (centre de la ville) et entrez la latitude et la longitude de ce point dans central_lat et central_lon respectivement

In [170]:
# Paramètres de recherche

# Filtre de recherche par localisation, exemple : localisation = "BREST MÉTROPOLE"
location_filter = "BREST MÉTROPOLE"

# FIltre de recherche par date, exemple : date_filter = "2021-01-01" dans le format "YYYY-MM-DD hh:mm:ss"
date_debut = "2020-01-01 00:00:00"
date_fin = "2021-12-31 23:59:59"

# Filtre des catégories à afficher, exemple : category_filter = "8" donne la catégorie Absence d'aménagement
category_filter = '8'

# Filtre des points d'intérêt à afficher, exemple : poi_filter = "Parking à vélo" donne le point d'intérêt Parking à vélo
poi_name = "Écoles"

# Boîte de recherche pour les points d'intérêt
central_lat = 48.390177
central_lon = -4.496499
delta = 0.02

bbox = [
    central_lat - delta,  # south latitude
    central_lon - delta,  # west longitude
    central_lat + delta,  # north latitude
    central_lon + delta   # east longitude
]

### Création du dataframe des données

In [171]:
def data_collection(location_filter, category_filter, date_debut=None, date_fin=None):
    try:
        # Construct the URL for the API call
        url = dict_association_localisation[location_filter] + 'c=' + category_filter
        resp = requests.get(url)
        resp.raise_for_status()  # Raises an exception for HTTP errors

        # Load data into DataFrame
        df = pd.DataFrame(resp.json())

        # Ensure 'time' column is treated as integer for Unix timestamp conversion
        # Explicitly cast to integer before conversion
        df['time'] = pd.to_datetime(df['time'].astype(int), unit='s')

        # Convert datetime back to Unix timestamp for comparison (if filtering is needed)
        if date_debut is not None and date_fin is not None:
            date_debut_dt = pd.to_datetime(date_debut).timestamp()
            date_fin_dt = pd.to_datetime(date_fin).timestamp()
            df = df[(df['time'].astype('int64') // 10**9 >= date_debut_dt) & 
                    (df['time'].astype('int64') // 10**9 <= date_fin_dt)]

        return df

    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return None
    except ValueError as e:
        print(f"Error parsing JSON or date conversion issue: {e}")
        return None
df = data_collection(location_filter, category_filter, date_debut, date_fin)

### Aperçu des données récoltées

In [172]:
df.head()

,token,coordinates_lat,coordinates_lon,address,comment,explanation,time,status,group,categorie,approved,cityname
27,A23C308D,48.3962633192363,-4.475417286157608,"Yves Collet, Brest",Carrefour dangereux - absence d'aménagement,"Le carrefour des rues Yves Collet, Kerivin, Sa...",2021-09-13 11:37:54,0,0,8,1,Brest
28,6B36F172,48.37977645614268,-4.54059261828661,"Rue Du Conquet, Brest","Rue neuve. Voie vélo❌, trottoir ❌, voiture✔",Où est l'espace réservé aux humains ?,2020-10-10 15:20:00,0,0,8,1,Brest
29,6FF86AF6,48.38163496867538,-4.506762586534023,"Rue Béranger , Brest","absence d'aménagement, d'arceau",,2020-09-21 14:39:00,0,0,8,1,Brest
30,F9482818,48.38038037995061,-4.526418820023537,"Rue Georges Bernanos, Brest",absence de piste cyclable,,2020-08-31 15:06:00,0,0,8,1,Brest
31,646DFA8B,48.38566,-4.46834,"Rue de Kiel, Brest",c'est très dangereux sans piste cyclable,,2020-08-26 15:02:00,0,0,8,1,Brest


### Récupération des données OpenStreetMap autour de la région cherchée

In [173]:
# Trier les données en fonction de leur coordonnées GPS, si elles sont dans la boîte de recherche autour du point d'intérêt
def fetch_poi_data(bbox, poi_name, poi_dict):
    # Fetch the tag for the poi_name from the poi_dict
    poi_tag = poi_dict.get(poi_name)
    if poi_tag is None:
        raise ValueError(f"Le point d'intérêt '{poi_name}' n'est pas reconnu.")
    
    # Format the bounding box and tags correctly for the Overpass query
    formatted_bbox = f"{bbox[0]},{bbox[1]},{bbox[2]},{bbox[3]}"
    formatted_poi_tag = poi_tag.replace("=", '="') + '"'

    # Define the Overpass API URL
    url = "http://overpass-api.de/api/interpreter"

    # Construct the query using the formatted bbox and poi_tag
    query = f"""
    [out:json];
    (
        node[{formatted_poi_tag}](bbox:{formatted_bbox});
        way[{formatted_poi_tag}](bbox:{formatted_bbox});
        rel[{formatted_poi_tag}](bbox:{formatted_bbox});
    );
    out center;
    """

    # Send the request to the Overpass API
    response = requests.get(url, params={'data': query})
    response.raise_for_status()  # Raises an HTTPError for bad responses
    return response.json()

def process_poi_data(json_data):
    # Initialize a dictionary to hold all POIs
    poi_dict = {}
    
    # Go through each element in the JSON data
    for element in json_data['elements']:
        # Check if the element is a node and has the required 'tags'
        if element['type'] == 'node' and 'tags' in element and 'amenity' in element['tags']:
                #Filter if the lat and lon are in the bbox again
                if bbox[0] <= element['lat'] <= bbox[2] and bbox[1] <= element['lon'] <= bbox[3]:
                    # Use the node ID as the key for the main dictionary
                    poi_id = element['id']
                    # Create a dictionary for this particular POI
                    poi_info = {
                        'name': element['tags'].get('name', 'Unknown'),
                        'latitude': element['lat'],
                        'longitude': element['lon'],
                    }
                    # Add this POI's information to the main dictionary
                    poi_dict[poi_id] = poi_info
    return poi_dict

pois = process_poi_data(fetch_poi_data(bbox, poi_name, poi_dict))
df_poi = pd.DataFrame.from_dict(pois, orient='index')
df_poi.head(15)

,name,latitude,longitude
727458152,Lycée général privé Sainte-Anne,48.396623,-4.490729
727458154,Lycée polyvalent et professionnel Jules Lesven,48.406575,-4.479869
727458160,Lycée La Pérouse-Kérichen,48.405368,-4.482298
727523418,École primaire privée de la Providence,48.396179,-4.482862
727523612,École primaire publique Kerhallet,48.402529,-4.516355
727544532,Collège privé Sainte-Anne,48.395836,-4.491997
727544535,Collège de la Croix Rouge,48.408144,-4.489686
3884466102,IBEP,48.404120,-4.512945
5789159196,École primaire de la Croix Rouge,48.407720,-4.489307


### Combinaison des données Vigilo et OpenStreetMap